## Results
Fit models to real-world data, as shown in @fig-fit.

In [ ]:
import warnings
from typing import List
import numpy as np
import pandas as pd
from datetime import datetime
import nevergrad as ng
from IPython.display import Markdown
from collections import namedtuple

from emu_renewal.renew import renew_taper_seed
from emu_renewal.distributions import get_gamma_densities_from_params
from emu_renewal.process import get_spline_interp_func, get_linear_interp_func, get_piecewise_cosine
from emu_renewal.outputs import plot_output_fit
from emu_renewal.markdown import get_param_table_from_instrum

In [ ]:
ModelResult = namedtuple('results', ['outputs', 'process', 'description'])

In [ ]:
def model_func(gen_time_mean: float, gen_time_sd: float, process_req: List[float], pop: int, seed: int, n_times: int, run_in: int) -> tuple:
    gen_desc = '\n\n### Generation times\n' \
        'Generation times for each day are calculated by ' \
        'first finding the parameters needed to construct ' \
        'a gamma distribution with mean and standard deviation ' \
        'equal to those specified by the submitted parameter values. ' \
        'The integrals of the probability density of this distribution ' \
        'between consecutive integer values are then calculated for ' \
        'later combination with the incidence time series. '
    gen_time_densities = get_gamma_densities_from_params(gen_time_mean, gen_time_sd, n_times)
    curve_desc = '\n\n### Non-mechanistic process\n' \
        'The sequence of parameter values pertaining to ' \
        "the model's non-mechanistic process are first exponentiated, " \
        'such that parameter exploration for these quantities is ' \
        'undertaken in the log-transformed space. ' \
        'Next, a continuous function of time was constructed from ' \
        'the non-mechanistic process series values submitted to the model. ' \
        'The fitting function consisted of a piecewise function ' \
        'constructed from a cosine function on domain zero to $\pi$. ' \
        'This function was then translated and scaled vertically and horizontally ' \
        'such that the starting and ending points of the cosine function ' \
        '(at which the gradient is zero) pass through the two consecutive ' \
        'process values. This provides a function of time for which ' \
        'the gradient and all higher order gradients are continuous. ' \
        'The time values corresponding to the submitted process values ' \
        'are set to be evenly spaced throughout the simulation period. '
    req_x_vals = np.linspace(0.0, n_times, len(process_req))
    func = get_piecewise_cosine(req_x_vals, process_req)
    process_vals = func(np.array([float(t) for t in range(n_times)]))
    process_vals_exp = np.exp(np.array(process_vals))
    renewal_output = renew_taper_seed(gen_time_densities, process_vals_exp, pop, np.exp(seed), n_times, run_in)
    return ModelResult(renewal_output, process_vals_exp, gen_desc + curve_desc + renewal_output.description)



In [ ]:
# Fixed parameters
population = 33e6
run_in = 30
n_process_periods = 12

In [ ]:
fixed_param_desc = '### Fixed parameter values\n ' \
    f'The target population is initialised as {str(int(population))} susceptible persons. ' \
    f'The simulation runs for a run-in period of {run_in} days before comparison against the calibration data commences.\n'

In [ ]:
raw_data = pd.read_csv('https://github.com/monash-emu/wpro_working/raw/main/data/new_cases.csv', index_col=0)['MYS']
raw_data.index = pd.to_datetime(raw_data.index)
mys_data = raw_data.loc[datetime(2021, 3, 1): datetime(2021, 11, 1)].reset_index()['MYS']
mys_data.index += run_in
n_times = len(mys_data) + run_in

In [ ]:
#| warning: false
calib_desc = '\n\n### Calibration targets\nThe model described above was fit to the target data ' \
    'to minimise the square of the difference between the modelled notification rate ' \
    'and the reported case numbers at each date considered. ' \
    'Modelled notifications is calculated as the product of modelled incidence and the ' \
    '(constant through time) case detection proportion. '
def calib_func(parameters: List[float], pop: int, n_times: int, run_in: int, targets: dict) -> float:
    gen_time_mean, gen_time_sd, cdr, seed, *process_req = parameters
    incidence = model_func(gen_time_mean, gen_time_sd, process_req, pop, seed, n_times, run_in).outputs.incidence
    return sum([(incidence[t] * cdr - d) ** 2 for t, d in targets.items()])

gen_time_mean_param = ng.p.Scalar(init=5.0, lower=0.1, upper=14.0).set_name('Generation time mean (days)')
gen_time_sd_param = ng.p.Scalar(init=5.0, lower=2.5, upper=8.0).set_name('Generation time standard deviation (days)')
cdr_param = ng.p.Scalar(init=0.06, lower=0.04, upper=0.2).set_name('Case detection proportion')
seed_param = ng.p.Scalar(init=np.log(1e4), lower=np.log(5e3), upper=np.log(2e4)).set_name('Log starting seed rate')
process_param = ng.p.Array(init=[0.0] * n_process_periods, lower=-2.0, upper=2.0).set_name('Log non-mechanistic process values')
instrum = ng.p.Instrumentation(gen_time_mean_param, gen_time_sd_param, cdr_param, seed_param, process_param)
def obj_func(gen_time_mean, gen_time_sd, cdr_param, seed, parameters):
    return calib_func([gen_time_mean, gen_time_sd, cdr_param, seed] + list(parameters), pop=population, n_times=n_times, run_in=run_in, targets=mys_data)
optimizer = ng.optimizers.NGOpt(parametrization=instrum, budget=100)
ngopt_result = optimizer.minimize(obj_func).value[0]

In [ ]:
#| label: fig-fit
#| fig-cap: "Optimisation to sample data from Malaysia"
model_result = model_func(ngopt_result[0], ngopt_result[1], ngopt_result[4], 33e6, ngopt_result[3], n_times, run_in)
renewal_desc = model_result.description
fig = plot_output_fit(mys_data, model_result.outputs, model_result.process, n_times, cdr=ngopt_result[2])
fig.write_image('results_fig.svg')

![Results of fitting to data from Malaysia.](results_fig.svg){#fig-fit}

## Methods

In [ ]:
Markdown(fixed_param_desc)

In [ ]:
Markdown(renewal_desc)

In [ ]:
Markdown(calib_desc)

### Calibrated parameters
The parameters used in the optimisation presented here are presented in @tbl-params.

In [ ]:
calib_params = get_param_table_from_instrum(instrum) 
Markdown(calib_params.to_markdown() + '\n : Parameters table {#tbl-params}')

In [ ]:
evidence_table = pd.DataFrame(index=calib_params.index, columns=['Evidence'])
evidence_table.loc[:, 'Evidence'] = 'To be populated [@cori2013]'
Markdown(evidence_table.to_markdown())

## References